##### How real-time questions can be solved. 
- We can create our own API. 
- We can extarnal API aswell. 

In [2]:
from openai import OpenAI
client = OpenAI(api_key=my_key)

In [5]:
prompt = "When's the next flight from lagos to navada?"

In [6]:
response = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=[{'role':'user','content':prompt}]
)

In [8]:
response.choices[0].message.content

"I'm sorry, but I am unable to provide real-time flight information. It is recommended to check with airlines or online flight booking platforms for the most up-to-date flight schedules and availability."

### Observations:
It's evident that the model:
- Does not have access to real-time information.
- Was trained on data up to 2021, which limits its ability to provide current information.

### Thoughts:
- Even if the model had information on flights, it cannot access or provide real-time data on constantly changing content.
- Integrating an external API can significantly enhance the functionality of the dialogue system, providing a more dynamic and user-friendly experience.

### Benefits of Using External APIs:
- Access to real-time data (e.g., flight information, weather updates, stock prices).
- Enhanced user interactions by providing up-to-date and relevant information.
- Expansion of functionalities, allowing for more complex and informative responses.

### Example Use Case:
Integrating a flight information API can help provide users with real-time updates on flight schedules, availability, and other relevant details, thereby improving the overall user experience.

### Future Implementation:
Consider implementing an external API to handle requests that require real-time data, and integrate the responses into the dialogue system for a seamless user experience.


=========================================================================================

In [9]:
function_description = [
    {
        'name': "get_flight_info",
        'description': "Get flight information between two locations",
        'parameters': {
            'type': 'object',
            'properties': {
                'loc_orign': {
                    'type': 'string',
                    'description': "The departure airport, e.g NOR"
                },
                'loc_destination': {
                    'type': 'string',
                    'description': "The destination airport, e.g BEL"
                },
            },
            "required": ["loc_orign", "loc_destination"]
        },
    }
]

In [10]:
response2 = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=[{'role':'user','content':prompt}],
    # this time i will be passing a function. 
    functions=function_description
)

In [12]:
response2.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"loc_orign":"LOS","loc_destination":"NVA"}', name='get_flight_info'), tool_calls=None)

In [13]:
response2.choices[0].message.function_call.arguments

'{"loc_orign":"LOS","loc_destination":"NVA"}'

Beautiful it was able to extract something from out function at list but we did not have a response. 

In [21]:
### lets create a random function to help. 
from datetime import datetime, timedelta
import json

def get_flight_info(loc_orign, loc_destination):

    flight_into = {
        'loc_orign': loc_orign,
        'loc_destination': loc_destination,
        'datetime': str(datetime.now() + timedelta(hours = 2)),
        'airline': "Britsh Airways",
        'flight': "BA678"
    }
    return json.dumps(flight_into)

In [22]:
# we are collecting it this way so we see later on
params = json.loads(response2.choices[0].message.function_call.arguments)

In [23]:
# this way i am actually getting what the true function is.
chosen_function = eval(response2.choices[0].message.function_call.name)

In [24]:
flight = chosen_function(**params)

flight 

'{"loc_orign": "LOS", "loc_destination": "NVA", "datetime": "2024-07-28 21:02:06.192764", "airline": "Britsh Airways", "flight": "BA678"}'

In [25]:
response3 = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=[
        {'role': 'user','content':prompt},
        {'role': 'function', 'name': response2.choices[0].message.function_call.name,
            'content':flight}
             ],
    # this time i will be passing a function. 
    functions=function_description
)

In [27]:
response3.choices[0].message.content

'The next flight from Lagos (LOS) to Navada (NVA) is scheduled for July 28, 2024. The airline is British Airways and the flight number is BA678.'